# Workflow as Agent with Reflection and Retry Pattern

## Overview

This notebook demonstrates an advanced **reflection and retry pattern** where:
1. A **Worker** executor generates responses to user queries
2. A **Reviewer** executor evaluates the quality using an LLM-based review
3. If not approved, the Worker regenerates the response incorporating feedback
4. This cycle continues until the Reviewer approves the output
5. Only approved responses are emitted externally via `AgentRunUpdateEvent`

The entire workflow is wrapped as an agent using `WorkflowAgent`, allowing it to be used like any standard agent with `run()` and `run_stream()` methods.

## Key Concepts Demonstrated

### 1. WorkflowAgent
- Wraps a workflow to behave like a regular agent
- Enables workflows to be composed as building blocks in larger systems
- Handles internal message routing while exposing standard agent interface

### 2. Cyclic Workflow Design
- **Worker ↔ Reviewer**: Bidirectional edges enable iterative improvement
- Unlike linear pipelines, cyclic workflows support feedback loops
- Workflow completes when idle (no pending messages)

### 3. AgentRunUpdateEvent
- Mechanism for emitting approved responses to external consumers
- Only approved outputs are surfaced outside the workflow
- Internal iterations remain hidden from the caller

### 4. Structured Output Parsing
- Uses Pydantic models with `response_format` parameter
- Ensures consistent, parseable feedback from the Reviewer
- Enables programmatic approval decisions

### 5. State Management
- Worker tracks pending requests via `_pending_requests` dictionary
- Correlates review responses with original requests using `request_id`
- Maintains conversation history across retry iterations

## Pipeline Layout

```
User Query
    ↓
Worker (generates response)
    ↓
Reviewer (evaluates quality)
    ↓ (if not approved)
Worker (regenerates with feedback) ──┐
    ↓                                 │
Reviewer (re-evaluates)               │
    └─────────────────────────────────┘
    ↓ (if approved)
AgentRunUpdateEvent (emit to external consumer)
```

## Prerequisites

- OpenAI API key configured (environment variable `OPENAI_API_KEY`)
- Agent Framework installed: `pip install agent-framework`
- Understanding of `WorkflowBuilder`, `Executor`, `WorkflowContext`, and event handling

## Setup and Imports

In [ ]:
import asyncio
from dataclasses import dataclass
from uuid import uuid4

import os
from dotenv import load_dotenv
from agent_framework import (
    AgentRunResponseUpdate,
    AgentRunUpdateEvent,
    ChatClientProtocol,
    ChatMessage,
    Contents,
    Executor,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    handler,
)
from agent_framework.openai import OpenAIChatClient
from pydantic import BaseModel
# Load environment variables from .env file
load_dotenv('../../.env')


## Define Message Types

### ReviewRequest
Structured request passed from Worker to Reviewer containing:
- `request_id`: Unique identifier for correlation
- `user_messages`: Original user query messages
- `agent_messages`: Worker's generated response messages

### ReviewResponse
Structured response from Reviewer back to Worker containing:
- `request_id`: Correlates with the original request
- `feedback`: Actionable guidance for improvement
- `approved`: Boolean indicating whether to emit or retry

In [ ]:
@dataclass
class ReviewRequest:
    """Structured request passed from Worker to Reviewer for evaluation."""
    request_id: str
    user_messages: list[ChatMessage]
    agent_messages: list[ChatMessage]


@dataclass
class ReviewResponse:
    """Structured response from Reviewer back to Worker."""
    request_id: str
    feedback: str
    approved: bool

## Create Reviewer Executor

The Reviewer uses an LLM to evaluate responses against quality criteria:
- **Relevance**: Does the response address the query?
- **Accuracy**: Is the information correct?
- **Clarity**: Is it easy to understand?
- **Completeness**: Does it cover all aspects?

### Key Implementation Details:

1. **Structured Output**: Uses Pydantic `BaseModel` with `response_format` to ensure parseable feedback
2. **Conversation Context**: Includes both user and agent messages for full context
3. **Review Instructions**: System prompt defines evaluation criteria
4. **Typed Context**: `WorkflowContext[ReviewResponse]` ensures type-safe message routing

In [ ]:
class Reviewer(Executor):
    """Executor that reviews agent responses and provides structured feedback."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client

    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext[ReviewResponse]) -> None:
        print(f"Reviewer: Evaluating response for request {request.request_id[:8]}...")

        # Define structured schema for the LLM to return.
        class _Response(BaseModel):
            feedback: str
            approved: bool

        # Construct review instructions and context.
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text=(
                    "You are a reviewer for an AI agent. Provide feedback on the "
                    "exchange between a user and the agent. Indicate approval only if:\n"
                    "- Relevance: response addresses the query\n"
                    "- Accuracy: information is correct\n"
                    "- Clarity: response is easy to understand\n"
                    "- Completeness: response covers all aspects\n"
                    "Do not approve until all criteria are satisfied."
                ),
            )
        ]
        # Add conversation history.
        messages.extend(request.user_messages)
        messages.extend(request.agent_messages)

        # Add explicit review instruction.
        messages.append(ChatMessage(role=Role.USER, text="Please review the agent's responses."))

        print("Reviewer: Sending review request to LLM...")
        response = await self._chat_client.get_response(messages=messages, response_format=_Response)

        parsed = _Response.model_validate_json(response.messages[-1].text)

        print(f"Reviewer: Review complete - Approved: {parsed.approved}")
        print(f"Reviewer: Feedback: {parsed.feedback}")

        # Send structured review result to Worker.
        await ctx.send_message(
            ReviewResponse(request_id=request.request_id, feedback=parsed.feedback, approved=parsed.approved)
        )

## Create Worker Executor

The Worker generates responses and incorporates feedback when necessary.

### Handler Methods:

#### 1. `handle_user_messages`
- Entry point for new user queries
- Generates initial response using chat client
- Creates `ReviewRequest` and sends to Reviewer
- Tracks request in `_pending_requests` for correlation

#### 2. `handle_review_response`
- Receives `ReviewResponse` from Reviewer
- If approved: Emits `AgentRunUpdateEvent` to external consumer
- If not approved: Incorporates feedback and regenerates response
- Creates new `ReviewRequest` for re-evaluation

### State Management:
- `_pending_requests`: Maps `request_id` to (ReviewRequest, conversation history)
- Enables correlation of review responses with original requests
- Maintains conversation context across retry iterations

In [ ]:
class Worker(Executor):
    """Executor that generates responses and incorporates feedback when necessary."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client
        self._pending_requests: dict[str, tuple[ReviewRequest, list[ChatMessage]]] = {}

    @handler
    async def handle_user_messages(self, user_messages: list[ChatMessage], ctx: WorkflowContext[ReviewRequest]) -> None:
        print("Worker: Received user messages, generating response...")

        # Initialize chat with system prompt.
        messages = [ChatMessage(role=Role.SYSTEM, text="You are a helpful assistant.")]
        messages.extend(user_messages)

        print("Worker: Calling LLM to generate response...")
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: Response generated: {response.messages[-1].text}")

        # Add agent messages to context.
        messages.extend(response.messages)

        # Create review request and send to Reviewer.
        request = ReviewRequest(request_id=str(uuid4()), user_messages=user_messages, agent_messages=response.messages)
        print(f"Worker: Sending response for review (ID: {request.request_id[:8]})")
        await ctx.send_message(request)

        # Track request for possible retry.
        self._pending_requests[request.request_id] = (request, messages)

    @handler
    async def handle_review_response(self, review: ReviewResponse, ctx: WorkflowContext[ReviewRequest]) -> None:
        print(f"Worker: Received review for request {review.request_id[:8]} - Approved: {review.approved}")

        if review.request_id not in self._pending_requests:
            raise ValueError(f"Unknown request ID in review: {review.request_id}")

        request, messages = self._pending_requests.pop(review.request_id)

        if review.approved:
            print("Worker: Response approved. Emitting to external consumer...")
            contents: list[Contents] = []
            for message in request.agent_messages:
                contents.extend(message.contents)

            # Emit approved result to external consumer via AgentRunUpdateEvent.
            await ctx.add_event(
                AgentRunUpdateEvent(self.id, data=AgentRunResponseUpdate(contents=contents, role=Role.ASSISTANT))
            )
            return

        print(f"Worker: Response not approved. Feedback: {review.feedback}")
        print("Worker: Regenerating response with feedback...")

        # Incorporate review feedback.
        messages.append(ChatMessage(role=Role.SYSTEM, text=review.feedback))
        messages.append(
            ChatMessage(role=Role.SYSTEM, text="Please incorporate the feedback and regenerate the response.")
        )
        messages.extend(request.user_messages)

        # Retry with updated prompt.
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: New response generated: {response.messages[-1].text}")

        messages.extend(response.messages)

        # Send updated request for re-review.
        new_request = ReviewRequest(
            request_id=review.request_id, user_messages=request.user_messages, agent_messages=response.messages
        )
        await ctx.send_message(new_request)

        # Track new request for further evaluation.
        self._pending_requests[new_request.request_id] = (new_request, messages)

## Build Workflow and Wrap as Agent

### Workflow Construction:

1. **Create chat clients**: Mini model for Worker, full model for Reviewer
2. **Create executors**: Worker and Reviewer instances
3. **Add bidirectional edges**: Worker ↔ Reviewer for cyclic feedback
4. **Set start executor**: Worker receives initial user messages
5. **Convert to agent**: `.as_agent()` wraps workflow with agent interface

### Key Points:

- **Bidirectional edges** enable the reflection pattern (Worker can send to Reviewer, Reviewer can send to Worker)
- **`.as_agent()`** allows the workflow to be used like any standard agent
- **Streaming execution** with `run_stream()` shows incremental updates

In [ ]:
async def run_reflection_workflow() -> None:
    print("Starting Workflow Agent Demo")
    print("=" * 50)

    # Initialize chat clients and executors.
    print("Creating chat client and executors...")
    mini_chat_client = OpenAIChatClient(model_id="gpt-4o-mini")
    chat_client = OpenAIChatClient(model_id="gpt-4o")
    reviewer = Reviewer(id="reviewer", chat_client=chat_client)
    worker = Worker(id="worker", chat_client=mini_chat_client)

    print("Building workflow with Worker ↔ Reviewer cycle...")
    agent = (
        WorkflowBuilder()
        .add_edge(worker, reviewer)  # Worker sends responses to Reviewer
        .add_edge(reviewer, worker)  # Reviewer provides feedback to Worker
        .set_start_executor(worker)
        .build()
        .as_agent()  # Wrap workflow as an agent
    )

    print("Running workflow agent with user query...")
    print("Query: 'Write code for parallel reading 1 million files on disk and write to a sorted output file.'")
    print("-" * 50)

    # Run agent in streaming mode to observe incremental updates.
    async for event in agent.run_stream(
        "Write code for parallel reading 1 million files on disk and write to a sorted output file."
    ):
        print(f"Agent Response: {event}")

    print("=" * 50)
    print("Workflow completed!")

## Run the Workflow

In [ ]:
await run_reflection_workflow()

## Expected Output

```
Starting Workflow Agent Demo
==================================================
Creating chat client and executors...
Building workflow with Worker ↔ Reviewer cycle...
Running workflow agent with user query...
Query: 'Write code for parallel reading 1 million files on disk and write to a sorted output file.'
--------------------------------------------------
Worker: Received user messages, generating response...
Worker: Calling LLM to generate response...
Worker: Response generated: Here's a Python implementation...
Worker: Sending response for review (ID: 12345678)
Reviewer: Evaluating response for request 12345678...
Reviewer: Sending review request to LLM...
Reviewer: Review complete - Approved: False
Reviewer: Feedback: The response lacks error handling and doesn't explain sorting mechanism.
Worker: Received review for request 12345678 - Approved: False
Worker: Response not approved. Feedback: The response lacks error handling...
Worker: Regenerating response with feedback...
Worker: Calling LLM to generate response...
Worker: New response generated: Here's an improved implementation with error handling...
Worker: Sending response for review (ID: 12345678)
Reviewer: Evaluating response for request 12345678...
Reviewer: Review complete - Approved: True
Reviewer: Feedback: All criteria met. Well-structured code with proper error handling.
Worker: Received review for request 12345678 - Approved: True
Worker: Response approved. Emitting to external consumer...
Agent Response: <AgentRunResponseUpdate with approved code>
==================================================
Workflow completed!
```

## Key Takeaways

### 1. Reflection Pattern Architecture
- **Cyclic workflows** enable iterative refinement through feedback loops
- Worker generates → Reviewer evaluates → Worker refines (repeat until approved)
- Only approved outputs are emitted to external consumers

### 2. WorkflowAgent Benefits
- Wraps complex internal workflows with simple agent interface
- Hides internal iterations from external callers
- Enables workflow composition and reuse

### 3. Structured Feedback
- Use Pydantic models with `response_format` for parseable LLM outputs
- Ensures consistent approval/rejection decisions
- Enables programmatic routing based on feedback

### 4. State Management
- Track pending requests using dictionaries keyed by `request_id`
- Correlate responses with original requests
- Maintain conversation history across iterations

### 5. Event Emission
- `AgentRunUpdateEvent` emits approved content to external consumers
- `ctx.add_event()` publishes events from within executors
- External callers only see final approved results

### 6. Production Considerations
- **Add iteration limits** to prevent infinite loops (e.g., max 3 retries)
- **Log all iterations** for debugging and analysis
- **Cache intermediate results** to avoid redundant LLM calls
- **Monitor costs** as each iteration incurs LLM API costs
- **Consider timeouts** for long-running review cycles

### 7. When to Use This Pattern
- **Quality-critical outputs**: Code generation, legal documents, medical advice
- **Multi-stage reasoning**: Complex queries requiring iterative refinement
- **Self-improvement**: Agents that learn from feedback
- **Compliance checks**: Automated validation against policies/standards